# 각각의 영상들을 30프레임의 160*160사이즈 넘파이 배열(.npy)로 저장하기
* 모델 훈련을 위한 트레인/테스트 셋을 만들 목적
* **`주의!`** **폭력 셋과 비폭력 셋을 따로 만드는 것이 좋습니다. 그래야 레이블 달기가 편합니다**
* **`결과물(피클) : 대용량 리스트 파일`** 입니다.
    * 개당 30GB 정도의 크기
    * D드라이브 등 피클 파일을 저장할 디스크의 용량이 확보되었는지 실행 전 확인해 주세요

* **`폭력 영상`** : 1814+5+22=1841개 / **`비폭력 영상`** : 1686+10+55=1741개
* **21.05.04. : ITWILL 촬영 영상 15개가 데이터셋에 새로 포함되었습니다.**
* **21.05.07. : ITWILL 쵤영 영상 77개가 데이터셋에 새로 포함되었습니다.**

* **`데이터셋 소스`** : 
    * RWF2000 : https://github.com/pedrofrodenas/Violence-Detection-CNN-LSTM/blob/master/violence_detection.ipynb
    * Hocky : http://visilab.etsii.uclm.es/personas/oscar/FightDetection/index.html
    * raw.zip : https://github.com/niyazed/violence-video-classification
    * cam1, cam2 : https://github.com/airtlab/A-Dataset-for-Automatic-Violence-Detection-in-Videos/tree/master/violence-detection-dataset/non-violent/cam1
    * 21.04.22., 21.05.04., 21.05.06. 학원에서 촬영한 동영상들 : 

# Imports

In [1]:
import os
import pickle
import numpy as np
from tqdm import tqdm
import cv2 #동영상을 불러오는 목적
from skimage.transform import resize # 이미지 리사이즈

# 영상을 30프레임의 160x160 넘파이 배열로 저장

In [2]:
def Save2Npy(file_dir, save_dir):
    """file_dir 폴더에 있는 비디오 영상을 불러와, Numpy 배열로 변환하여 save_dir 폴더에 저장합니다.
    ::Args::
    file_dir: 원본 비디오들이 있는 폴더
    save_dir: 넘파이 배열로 변환한 결과인 .npy 파일들을 저장할 폴더
    """
    if not os.path.exists(save_dir): # save_dir 폴더가 없으면, 그 위치에 새 폴더를 만들어 줍니다.
        os.makedirs(save_dir)
    
    file_list=os.listdir(file_dir) # file_dir 폴더 내에 존재하는 파일들(동영상 이름들) 리스트화
    
    for file in tqdm(file_list):
        frames=np.zeros((30, 160, 160, 3), dtype=np.float)
        i=0
        
        vc=cv2.VideoCapture(os.path.join(file_dir, file)) # file_dir 내의 동영상(file)들에 대해 비디오 객체 생성
        
        if vc.isOpened():
            rval, frame=vc.read()
        else:
            rval=False
            
        frm=resize(frame, (160, 160, 3))
        frm=np.expand_dims(frm, axis=0)
        
        if(np.max(frm)>1):
            frm=frm/255.0
        frames[i][:]=frm
        i+=1
      
        while i<30:
            rval, frame=vc.read()
            frm=resize(frame,(160,160,3))
            frm=np.expand_dims(frm,axis=0)
            if(np.max(frm)>1):
                frm=frm/255.0
            frames[i][:] = frm
            i+=1

        video_name=file.split('.')[0]
        video_path=os.path.join(file_dir, file)
        save_path=os.path.join(save_dir, video_name+'.npy')

        np.save(save_path, frames)

In [3]:
file_dir='D:/datasets/itwill/Fight' # 영상이 있는 폴더
save_dir='D:/datasets/AllVideo_Fight_Numpy' # 각 영상별 30프레임의 넘파이를 저장할 폴더

In [4]:
Save2Npy(file_dir=file_dir, save_dir=save_dir)

100%|██████████████████████████████████████████████████████████████████████████████████| 27/27 [01:16<00:00,  2.82s/it]


In [5]:
file_dir='D:/datasets/itwill/NonFight' # 영상이 있는 폴더
save_dir='D:/datasets/AllVideo_NonFight_Numpy' # 각 영상별 30프레임의 넘파이를 저장할 폴더

In [6]:
Save2Npy(file_dir=file_dir, save_dir=save_dir)

100%|██████████████████████████████████████████████████████████████████████████████████| 55/55 [02:56<00:00,  3.21s/it]


# 저장된 넘파이 배열들을 리스트에 추가하여 데이터 셋(리스트) 만들기

## 폭력 영상 : Fight 

In [7]:
Fight_dir='D:/datasets/AllVideo_Fight_Numpy' # 영상 넘파이 배열들이 있는 폴더
file_list_npy = os.listdir(Fight_dir) # 넘파이 배열들이 저장된 폴더(save_dir) 내의 파일이름들

data_Fight=[]
for file in file_list_npy: # 해당 폴더 내의 전체 넘파이들에 대해서
    file_path=os.path.join(Fight_dir, file)
    x=np.load(file_path)
    data_Fight.append(x)

print(len(data_Fight))

1841


## 비폭력 영상 : NonFight

In [8]:
NonFight_dir='D:/datasets/AllVideo_NonFight_Numpy' # 영상 넘파이 배열들이 있는 폴더
file_list_npy = os.listdir(NonFight_dir) # 넘파이 배열들이 저장된 폴더(save_dir) 내의 파일이름들

data_NonFight=[]
for file in file_list_npy: # 해당 폴더 내의 전체 넘파이들에 대해서
    file_path=os.path.join(NonFight_dir, file)
    x=np.load(file_path)
    data_NonFight.append(x)

print(len(data_NonFight))

1741


# 만들어진 데이터셋(리스트)을 피클로 저장

In [9]:
with open("D:/datasets/AllVideo_numpy_list_pickle/01_data_Fight_210507.pickle","wb") as fw:
    pickle.dump(data_Fight, fw, protocol=pickle.HIGHEST_PROTOCOL)

In [10]:
with open("D:/datasets/AllVideo_numpy_list_pickle/01_data_NonFight_210507.pickle","wb") as fw:
    pickle.dump(data_NonFight, fw, protocol=pickle.HIGHEST_PROTOCOL)

# 레이블(실제값) 저장

> 다음에 유의
```
prediction=preds.argmax(axis=0):[0 0]
Results =  [[0.09166703 0.9092251 ]]
Maximun Probability =  0.9092251
Difference of prob  0.8184502124786377
Alert : violence - 90.92%
```
> labels1=[] 각 비디오의 레이블(폭력 / 비폭력). 
> * 폭력 : [0,1]
> * 비폭력 : [1,0]

In [11]:
label_Fight_per_video=np.array([0,1])
label_Fight=[label_Fight_per_video]*len(data_Fight) #폭력 영상 개수만큼

In [12]:
label_NonFight_per_video=np.array([1,0])
label_NonFight=[label_NonFight_per_video]*len(data_NonFight) #비폭력 영상 개수만큼

In [13]:
len(label_Fight), len(label_NonFight)

(1841, 1741)

In [14]:
label_Fight[0], label_NonFight[0]

(array([0, 1]), array([1, 0]))

In [15]:
with open("D:/datasets/AllVideo_numpy_list_pickle/01_label_Fight_210507.pickle","wb") as fw:
    pickle.dump(label_Fight, fw)

In [16]:
with open("D:/datasets/AllVideo_numpy_list_pickle/01_label_NonFight_210507.pickle","wb") as fw:
    pickle.dump(label_NonFight, fw)